In [2]:
#!pip install matplotlib==3.7.0

In [1]:
import os 
import torch 
import matplotlib 
import functions 

cwd = os.getcwd()
cwd

'C:\\Users\\Utente\\Desktop\\ECG_SR'

In [2]:
labels = ["MI", "STTC", "CD", "HYP", "NORM"]

In [3]:
dict_windows_test_lr_n = {}
path = os.getcwd()+os.sep+"pt_data"+os.sep

for label in labels:
    
    filename = path + "test_{}_noisy.pt".format(label)
    lr_data = torch.load(filename)
    dict_windows_test_lr_n[label] = lr_data

In [26]:
for label, values in dict_windows_test_lr_n.items():

    print(label, values.shape)

MI torch.Size([1088, 1, 12, 250])
STTC torch.Size([568, 1, 12, 250])
CD torch.Size([490, 1, 12, 250])
HYP torch.Size([244, 1, 12, 250])
NORM torch.Size([1926, 1, 12, 250])


In [4]:
x_test = torch.cat(list(dict_windows_test_lr_n.values()))
x_test.shape

torch.Size([4316, 1, 12, 250])

In [5]:
x_test = torch.unsqueeze(x_test, dim = 1)
x_test.shape

torch.Size([4316, 1, 1, 12, 250])

In [13]:
device = "cuda:0"
x_test.to(device)

tensor([[[[[ 3.2185e-02, -4.9509e-02, -2.0301e-02,  ...,  2.0161e-02,
            -3.0261e-02,  2.0745e-02],
           [ 5.6784e-02, -6.0794e-02, -5.2772e-03,  ..., -1.5538e-02,
            -7.1191e-02, -1.0413e-02],
           [ 2.4866e-02, -1.1050e-02,  1.5306e-02,  ..., -3.5009e-02,
            -4.1108e-02, -3.0697e-02],
           ...,
           [ 6.6937e-02, -9.6021e-02,  1.9699e-03,  ...,  1.2915e-01,
             7.9756e-02,  1.2479e-01],
           [ 1.8587e-02, -1.2386e-01, -2.3833e-02,  ..., -9.0272e-02,
            -1.0182e-01, -4.4736e-02],
           [ 7.5363e-03, -1.1401e-01, -3.2593e-02,  ..., -1.7279e-01,
            -1.8284e-01, -1.1000e-01]]]],



        [[[[ 5.3926e-03,  6.2761e-02,  5.9772e-02,  ...,  6.9501e-02,
             7.9347e-02,  1.2017e-01],
           [-3.8360e-02,  9.7371e-03,  2.2684e-02,  ...,  4.3259e-02,
             7.1064e-02,  1.4193e-01],
           [-4.3699e-02, -5.2628e-02, -3.7565e-02,  ..., -2.6185e-02,
            -8.2508e-03,  2.1690e-02

In [15]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [17]:
sep = os.sep 

device = "cuda:0"
modelspath = cwd + sep + "models"
modelpath = modelspath + sep + "model_2_denoising_lrhr_v2.pt"

In [19]:
model = torch.load(modelpath).to(device)
model.eval()

decoder = model.decoder.decoder
decoder_sr = model.upsample.decoder
for j, block in enumerate(decoder):
  if j+1 == 2:
    block.last_tanh = False
  else:
    block.last_tanh = True

for j, block in enumerate(decoder_sr):
  if j+1 == 2:
    block.last_tanh = False
  else:
    block.last_tanh = True

D:\anaconda3\envs\dcaesr\lib\site-packages\torch\serialization.py:1172: UserWarning: This figure was saved with matplotlib version 3.7.1 and is unlikely to function correctly.
  result = unpickler.load()


In [21]:
import time 

times = []

for signal_lr in x_test:
    
    start = time.time()
    signal_sr = model([signal_lr, None, None, None])[1]
    t = time.time() - start
    times.append(t)

In [28]:
import numpy as np 
np.mean(times), np.sum(times)

(0.013920831779730993, 60.08230996131897)